In [7]:
import pandas as pd
import numpy as np
import tweepy
import datetime
import time
import pickle
import tqdm
from src.tools.twitter_api2 import auth

In [8]:
with open('../Data/Processed/Usr_ID_dict.pickle', 'rb') as handle:
    Usr_ID = pickle.load(handle)

ID_Usr = {I: U for U, I in Usr_ID.items()}

Handle_data = pd.read_csv('../Data/Processed/Twitter_Handles.csv')
Handle_dict = pd.Series(Handle_data.Name.values, index=Handle_data.Twitter.values).to_dict()

In [9]:
Follow_df = pd.DataFrame(data = 0, index = Handle_dict.keys(), columns = Handle_dict.keys())
No_follow_dict = dict.fromkeys(Handle_dict.keys())

In [10]:
def get_following(name):
    try:
        ids_list = []
        for page in tweepy.Cursor(api.friends_ids, screen_name=name).pages():
            ids_list.extend(page)

        return len(ids_list), [ID_Usr[id] for id in ids_list if id in ID_Usr.keys()]
    except Exception:
        time.sleep(60) # Wait 1 minutes if limit is reached or another error is encounted
        get_following(name)

In [20]:
api = tweepy.API(auth, wait_on_rate_limit=True)
for name in tqdm.tqdm(Follow_df.index[385:]):
    # Try again if fail (which happens first time after the api values are replenished)
    try: 
        no_follows, Follows = get_following(name)
    except TypeError:
        no_follows, Follows = get_following(name)

    # Add results
    No_follow_dict[name] = no_follows
    Follow_df.loc[name,Follows] = 1

100%|██████████| 233/233 [12:24:36<00:00, 191.74s/it]


In [23]:
Follow_df.to_csv('../Data/Processed/Follow_df.csv')

In [24]:
with open('../Data/Processed/No_follow_dict.pickle', 'wb') as handle:
    pickle.dump(No_follow_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
Follow_df.sum(axis=1).sort_values()

SenJonKyl          0
MikeforWI          0
YvetteClarke       0
RepTomMarino       0
RepHensarling      0
                ... 
RepTrentKelly    460
RepGolden        472
repmarkpocan     482
RepCartwright    486
justinamash      537
Length: 618, dtype: int64